In [38]:
import pandas as pd
import pickle

In [39]:
def save_dict(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_dict(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [40]:
df = pd.read_csv("./msf_comcare.csv", header = None)

In [41]:
questions = []
lb = 0
answers = []
for q,a in zip(df[1], df[2]):
    questions.append(q)
    answers.append(a)

In [42]:
answers_unique = list(set(answers)) 

In [43]:
ans_map = dict()
for i,ans in enumerate(answers_unique):
    ans_map[ans] = i
    

In [44]:
labels = []
for i in range(len(answers)):
    labels.append(ans_map[answers[i]])

In [45]:
comcare_questions = {q:l for q,l in zip(questions,labels)}

In [51]:
data = {"question_to_label" : comcare_questions, "answer_to_label" : ans_map}

In [54]:
save_dict(data,"./comcare_orignal.pkl")